In [6]:
from flask import Flask, render_template,request
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
import pandas_ta as ta
import plotly.io as pio
pio.renderers.default = 'vscode'
from sklearn.cluster import DBSCAN
import json

ImportError: cannot import name 'NaN' from 'numpy' (/Users/adrish_mitra/anaconda3/envs/spurzee/lib/python3.12/site-packages/numpy/__init__.py)

In [ ]:
from fyers_apiv3 import fyersModel
import os
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
# from fyers_api.Websocket import data_ws
with open("abcd.txt", 'r') as r:
    access_token = r.read()
client_id = "DZO41L3M36-100"
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'YL00137',
  'name': 'LOKESH TALLURI',
  'image': None,
  'display_name': None,
  'pin_change_date': '31-12-2023 10:51:16',
  'email_id': 'tallurilokesh2003@gmail.com',
  'pwd_change_date': None,
  'PAN': '---------',
  'mobile_number': '9515652294',
  'totp': True,
  'pwd_to_expire': 90}}

In [ ]:
with open("50stocks.txt","r") as f:
    lines = f.readlines()

In [ ]:
import pandas as pd
import numpy as np
# import ta
# from fyers_api import fyersModel
from sklearn.cluster import KMeans
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)

time_intervals = [
    {'label': '5 mins', 'value': '5'},
    {'label': '15 mins', 'value': '15'},
    {'label': '30 mins', 'value': '30'},
    {'label': '60 mins', 'value': '60'}
]


def generate_graph(symbol, start_date, end_date, interval):
    data = {
        "symbol": symbol,
        "resolution": interval,
        "date_format": "1",
        "range_from": start_date,
        "range_to": end_date,
        "cont_flag": "1"
    }
    df = pd.DataFrame(fyers.history(data=data)['candles'])
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.date = (df.date.dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata'))
    df['date'] = df['date'].dt.tz_localize(None)
    df["EMA"] = ta.ema(df["close"], length=20)

    sup = df[df.low == df.low.rolling(20, center=True).min()].low
    res = df[df.high == df.high.rolling(20, center=True).max()].high
    lev = sup.tolist() + res.tolist()
    lev.sort()

    kmeans = KMeans(n_clusters=min(5, len(lev))).fit(
        np.array(lev).reshape(-1, 1))
    lset = []
    for cluster_center in kmeans.cluster_centers_:
        closest_index = np.argmin(np.abs(lev - cluster_center))
        lset.append(lev[closest_index])

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.1, row_heights=[0.8, 0.2])
    fig.add_trace(go.Candlestick(x=df['date'], open=df['open'], high=df['high'],
                  low=df['low'], close=df['close'], name='Candlestick'), row=1, col=1)
    ema_trace = go.Scatter(x=df['date'], y=df['EMA'],
                           mode='lines', name='EMA', visible='legendonly')
    fig.add_trace(ema_trace, row=1, col=1)
    fig.add_trace(go.Bar(x=df['date'], y=df['volume'],
                  name='Volume', marker_color='blue', marker_opacity=0.9), row=2, col=1)
    fig.update_layout(
        title=symbol, xaxis_rangeslider_visible=False, showlegend=False)

    button_layer_1_height = 1.19
    fig.update_layout(
        updatemenus=[
            dict(
                buttons=list([
                    dict(label="Show EMA",
                         method="update",
                         args=[{"visible": [True]}]
                         ),
                    dict(label="Hide EMA",
                         method="update",
                         args=[{"visible": [True, 'legendonly']}]
                         ),
                ]),
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.06,
                xanchor="left",
                y=button_layer_1_height,
                yanchor="top"
            ),
        ]
    )

    fig.update_layout(
        annotations=[
            dict(text="EMA", x=0, xref="paper", y=1.111, yref="paper",
                 align="left", showarrow=False),
            dict(text="Interval(mins)", x=0.2, xref="paper", y=1.111,
                 yref="paper", showarrow=False),
        ])

    fig.update_xaxes(type='category', tickmode='auto',
                     nticks=6, tickformat='%Y-%m-%d %H:%M:%S')
    fig.update_yaxes(side='left')

    for l in lset:
        fig.add_shape(type='line', x0=0, y0=l, x1=len(df), y1=l)

    return fig


symbol = "NSE:NIFTY50-INDEX"
start_date = "2024-01-01"
end_date = "2024-03-14"
interval = "15"
graph = generate_graph(symbol, start_date, end_date, interval)

# app.layout = html.Div([
#     dcc.Graph(id='graph', figure=graph, style={'width': '80%', 'display': 'inline-block'}),
#     html.Div([
#         dcc.Dropdown(
#             id='interval-dropdown',
#             options=time_intervals,
#             value=interval,
#             style={'width': '150px'}
#         )
#     ], style={'position': 'absolute', 'top': '67px', 'right': '780px'})
# ])
app.layout = html.Div([
    dcc.Graph(id='graph', figure=graph, style={'width': '70%', 'display': 'inline-block'}),
    html.Div([
        dcc.Dropdown(
            id='interval-dropdown',
            options=time_intervals,
            value=interval,
            style={'width': '150px'}
        )
    ], style={'position': 'absolute', 'top': '67px', 'right': '780px'}),
    
    html.Div([
        html.Div(id='graph-info-box',children=[
        html.Div(
            id='custom-menu',
            children=[
                html.Div(className='custom-menu-option', children='NSE:NIFTY50-INDEX', id='NSE:NIFTY50-INDEX',**{'data-value': 1}),
                html.Div(className='custom-menu-option', children='NSE:NIFTYBANK-INDEX', id='NSE:NIFTYBANK-INDEX',**{'data-value': 2}),
                html.Div(className='custom-menu-option', children='MCX:GOLD24MAR55000CE', id='MCX:GOLD24MAR55000CE',**{'data-value': 3}),
                html.Div(className='custom-menu-option', children='NSE:RELIANCE-EQ', id='NSE:RELIANCE-EQ',**{'data-value': 4}),
                
            ],
            style={
                'display': 'flex',
                'flex-direction': 'column',
                 'width' : '100%',
                'height': '100%', 
                'font-size': '15px', 
                'margin': '0px',
                'padding': '15px', 
                'border': 'none',
                'background-color': 'transparent'
            }
        )
    ], style={'border': '2px solid #ccc', 'padding': '20px', 'position': 'fixed',  # Make the box fixed
        'top': '100px',  # Adjust top position from the edge
        'right': '50px',  # Position at right corner
        'width': '27%',
        'height': '150px',  # Set desired height
        'display': 'flex',  # Allow content to align horizontally
        'align-items': 'center',  # Center content vertically
        'justify-content': 'center', })
    ])
])
@app.callback(
    Output('graph', 'figure'),
    [Input('interval-dropdown', 'value')]
)
def update_graph(interval):
    # Call generate_graph function with updated interval
    return generate_graph(symbol, start_date, end_date, interval)


if __name__ == '__main__':
    app.run_server(debug=True)

SyntaxError: invalid syntax (4190147519.py, line 137)

In [ ]:
# from flask import Flask, render_template
# app = Flask(__name__, template_folder='temp')


# @app.route('/')
# def index():
#     fig = graph

#     graph_json = fig.to_json()

#     return render_template('graph.html', graph_json=graph_json)


# if __name__ == '__main__':
#     app.run(debug=True,use_reloader=False)

In [ ]:
# for line in lines:
#     line = line.strip()
#     data = {
#         "symbol":line,
#         "resolution":"60",
#         "date_format":"1", 
#         "range_from":"2024-01-02",
#         "range_to":"2024-02-12",
#         "cont_flag":"1"
#     }
#     df = fyers.history(data=data)
#     df=pd.DataFrame(df['candles'])
#     df.columns=['date','open','high','low','close','volume']
#     df['date']=pd.to_datetime(df['date'],unit='s')
#     df.date=(df.date.dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata'))
#     df['date']=df['date'].dt.tz_localize(None)
#     print(line)
#     sup=df[df.low==df.low.rolling(20,center=True).min()].low
#     res=df[df.high==df.high.rolling(20,center=True).max()].high
#     sup=sup.tolist()
#     res=res.tolist()
#     lev=sup+res
#     lev.sort()
#     channel=15
#     cwidth=(df['high'].max()-df['low'].min())*channel/100
#     sr_zones = get_sr_zones(lev, cwidth)
#     max_num_sr = 6
#     min_strength = 2
#     sr_up_level = []
#     sr_dn_level = []
#     sr_strength = []
#     for zone in sr_zones:
#         if zone['strength'] >= min_strength:
#             loc = find_loc(zone['strength'], sr_strength)
#             if loc < max_num_sr and check_sr(zone['high'], zone['low'], zone['strength'], sr_up_level, sr_dn_level, sr_strength):
#                 sr_strength.insert(loc, zone['strength'])
#                 sr_up_level.insert(loc, zone['high'])
#                 sr_dn_level.insert(loc, zone['low'])
#                 if len(sr_strength) > max_num_sr:
#                     sr_strength.pop(0)
#                     sr_up_level.pop(0)
#                     sr_dn_level.pop(0)
#     lset=[(sr_up_level[i]+sr_dn_level[i])/2 for i in range(len(sr_up_level))]
    
#     fig = go.Figure(data=[go.Candlestick(x=df.index,
#                     open=df['open'],
#                     high=df['high'],
#                     low=df['low'],
#                     close=df['close'])])
#     for l in lset:
#         fig.add_shape(type='line', x0=0, y0=l,
#                       x1=300, 
#                       y1=l
#                       )
#     fig.show()
#     break



In [ ]:
sup=df[df.low==df.low.rolling(20,center=True).min()].low
res=df[df.high==df.high.rolling(20,center=True).max()].high
sup=sup.tolist()
res=res.tolist()

NameError: name 'df' is not defined

In [ ]:
def get_sr_zones(pivot_vals, max_width):
    sr_zones = []
    for i in range(len(pivot_vals)):
        lo = hi = pivot_vals[i]
        num_pp = 0
        for j in range(len(pivot_vals)):
            cpp = pivot_vals[j]
            width =  hi - cpp if cpp <= lo else cpp - lo  # Calculate width from high
            if width <= max_width:
                if cpp <= hi:
                    lo = min(lo, cpp)
                else:
                    hi = max(hi, cpp)
                num_pp += 1
        sr_zones.append({'high': hi, 'low': lo, 'strength': num_pp})
    return sr_zones
def find_loc(strength, sr_strength):
    loc = len(sr_strength)
    for i in range(len(sr_strength) - 1, -1, -1):
        if strength <= sr_strength[i]:
            break
        loc = i
    return loc

def check_sr(hi, lo, strength, sr_up_level, sr_dn_level, sr_strength):
    for i in range(len(sr_up_level)):
        if sr_up_level[i] >= lo and sr_up_level[i] <= hi or sr_dn_level[i] >= lo and sr_dn_level[i] <= hi:
            if strength >= sr_strength[i]:
                return False
    return True


In [ ]:
lev=sup+res
lev.sort()

dbscan = DBSCAN(eps=150, min_samples=5).fit(np.array(lev).reshape(-1, 1))
    
    # Extract cluster labels
labels = dbscan.labels_

# Get unique cluster labels (excluding noise points)
unique_labels = np.unique(labels[labels != -1])

# Initialize list to store nearest points
lset = []

# Find nearest point within each cluster
for label in unique_labels:
    cluster_points = np.array(lev)[labels == label]
    cluster_center = np.mean(cluster_points)
    nearest_index = np.argmin(np.abs(cluster_points - cluster_center))
    lset.append(cluster_points[nearest_index])
lset

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
for l in lset:
    fig.add_shape(type='line', x0=0, y0=l,
                  x1=210, 
                  y1=l
                  )

In [ ]:
fig

In [ ]:
import numpy as np
channel=15
cwidth=(df['high'].max()-df['low'].min())*channel/100
sr_zones = get_sr_zones(lev, cwidth)
print(sr_zones)
max_num_sr = 6
min_strength = 2
sr_up_level = []
sr_dn_level = []
sr_strength = []
for zone in sr_zones:
    if zone['strength'] >= min_strength:
        loc = find_loc(zone['strength'], sr_strength)
        if loc < max_num_sr and check_sr(zone['high'], zone['low'], zone['strength'], sr_up_level, sr_dn_level, sr_strength):
            sr_strength.insert(loc, zone['strength'])
            sr_up_level.insert(loc, zone['high'])
            sr_dn_level.insert(loc, zone['low'])
            if len(sr_strength) > max_num_sr:
                sr_strength.pop(0)
                sr_up_level.pop(0)
                sr_dn_level.pop(0)


In [ ]:
lset=[(sr_up_level[i]+sr_dn_level[i])/2 for i in range(len(sr_up_level))]
lset
      

In [ ]:

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
for l in lset:
    fig.add_shape(type='line', x0=0, y0=l,
                  x1=700, 
                  y1=l
                  )


In [ ]:
fig